In [ ]:
#Noun+Verb LemSpacy LDA GBM: 0.8071395816658974

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim import corpora
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = stopwords.words('english')
sp = spacy.load('en_core_web_lg')

categories = ['rec.autos', 'comp.graphics', 'sci.space']
newsgroup = fetch_20newsgroups(categories=categories, shuffle=True)
data = newsgroup.data
target = newsgroup.target

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhest\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def lemmatizeSpacy(input):
    doc = sp(input)
    #Tokenizers divide strings into lists of substrings
    output = ' '.join([w.lemma_ for w in doc])
    return output

In [3]:
def NVExt(input):
    output=""
    doc = sp(input)
    for w in doc:
        if w.pos_ in ["NOUN","VERB"]:
            output+=w.text+" "
    output = output[:-1]
    return output

In [4]:
def train(x_train, x_test, y_train, y_test, n):
    clf = GradientBoostingClassifier(n_estimators=n)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    return f1_score(y_test, y_pred, average='weighted')

In [5]:
def textProcessing(unprocessedTexts, function=None):
    if function is None:
        return unprocessedTexts
    return [function(text) for text in unprocessedTexts]

In [6]:
dataExt = textProcessing(data,NVExt)
dataProc = textProcessing(dataExt,lemmatizeSpacy)
tokenized_documents = [simple_preprocess(text) for text in dataProc]
dictionary = corpora.Dictionary(tokenized_documents)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [7]:
alphaArr = ['auto',0.00001,0.0001,0.001,0.01,0.1,0.2,0.4,0.5,0.7,0.9,1,1.2,1.4,1.6,1.8,2,4,6,8,10,14,16,18,20,25,30]
betaArr = ['auto',0.00001,0.0001,0.001,0.01,0.1,0.2,0.4,0.5,0.7,0.9,1,1.2,1.4,1.6,1.8,2,4,6,8,10,14,16,18,20,25,30]
numArr = [3,5,7,10,15,20,25,30,35,40,50,60,70,80,90,100,120,140,160,180,200,225,250,275,300,350,400]
mArr = [10,20,30,40,50,60,70,80,90,100,120,140,160,180,200,225,250,275,300,350,400,450,500]
list = []
for alp in alphaArr:
    for beta in betaArr:
        for num in numArr:
            for m in mArr:
                length = num
                lda = LdaModel(bow_corpus, num_topics=num, id2word=dictionary, alpha=alp, eta=beta)
                dataVecPre = lda[bow_corpus]
                len_data = len(dataVecPre)
                dataVec = [0 for o in range(len_data)]
                for o in range(len_data):
                    curr = []
                    a = dataVecPre[o]
                    lenCurr = len(a)
                    if length!=lenCurr:
                        curr = [0 for u in range(length)]
                        c = 0
                        for u in range(length):
                            if c>=lenCurr:
                                curr[u] = 0
                            else:
                                if a[c][0]!=u:
                                    curr[u] = 0
                                else:
                                    curr[u] = a[c][1]
                                    c+=1
                        dataVec[o] = curr
                    else:
                        curr = [element[1] for element in a]
                        dataVec[o] = curr
                dataVec = np.array(dataVec)
                X_train, X_test, y_train, y_test = train_test_split(dataVec, target, test_size=0.33)
                f1 = train(X_train, X_test, y_train, y_test, m)
                list.append([f1,alp,beta,num,m])
maxList = [w[0] for w in list]
maximum = max(maxList)
print(maximum)
print(list[maxList.index(maximum)])

KeyboardInterrupt: 